$$ \max Z = 3x_1  + 2x_2 + 5x_3$$


\begin{align*}
x_1 + 2x_2 + x_3 &\leq 430\\
3x_1 + 2x_3 &\leq 460\\
x_1 + 4x_2 &\leq 420\\[4mm]
x_1, x_2, x_3 &\geq 0
\end{align*}
 


Encuentre la solución óptima. Suponga que los coeficientes del lado izquierdo (coeficientes tecnológicos) de  la columna $x_1$ cambian de $[1 \;,\; 3 \;,\; 1]^{T}$ a $[1 \;,\; 1 \;,\; 6]^{T}$ en la matriz inicial y los coeficientes de $x_2$ and $x_3$ en la función objetivo cambian de $[2\;, \; 5]$ a $[1\;,\; 3]$ **encuentre la nueva solución óptima.**


In [1]:
from fractions import Fraction

import numpy as np
import pandas as pd

np.set_printoptions(formatter={'all': lambda x: str(Fraction(x).limit_denominator())})

In [257]:
%run simplex_algorithm.ipynb

In [92]:
left_hand_coefficients = np.array([
    [1, 2, 1, 1, 0, 0],
    [3, 0, 2, 0, 1, 0],
    [1, 4, 0, 0, 0, 1],
    
])

right_hand_side = np.array([430, 460, 420])

zvector = np.array([3, 2, 5, 0, 0, 0])

In [93]:
table = simplex(matrix=left_hand_coefficients, rhs=right_hand_side, z=zvector, direction=1)

========== Simplex Method ==========
Optimal solution found in 2 iterations
Optimal Table: 
 [[-1/4 1 0 1/2 -1/4 0]
 [3/2 0 1 0 1/2 0]
 [2 0 0 -2 1 1]] 

Solution [0 100 230 0 0 20] 	Z: 1350.00
========== Simplex Method ==========


In [94]:
print(table)

      x1   x2   x3   x4    x5   x6      b   cb  cbidx
x2 -0.25  1.0  0.0  0.5 -0.25  0.0  100.0  2.0    1.0
x3  1.50  0.0  1.0  0.0  0.50  0.0  230.0  5.0    2.0
x6  2.00  0.0  0.0 -2.0  1.00  1.0   20.0  0.0    5.0


$$\mathbf{B} = \begin{bmatrix} x_2 & x_3& s_3 \end{bmatrix}$$

In [201]:
cbindex = table['cbidx'].values.astype(int)


B = left_hand_coefficients[:, cbindex]
print(B)

[[2 1 0]
 [0 2 0]
 [4 0 1]]


In [202]:
Binv = np.linalg.inv(B)
print(Binv)

[[1/2 -1/4 0]
 [0 1/2 0]
 [-2 1 1]]


La matrix base $\mathbf{B}$ es $$\mathbf{B^{-1}} = 
\begin{bmatrix}
x_2 & x_3 & s_3\\
2 & 1 & 0 \\
0 & 2 & 0 \\
4 & 0 & 1 \\
\end{bmatrix}
$$
Por lo tanto la matriz inversa de $\mathbf{B}$ es
$$\mathbf{B^{-1}} = 
\begin{bmatrix}
\frac{1}{2} & -\frac{1}{4} & 0 \\
0 & \frac{1}{2} & 0 \\
-2 & 1 & 1 \\
\end{bmatrix}
$$

Los rangos de $\Delta_1, \Delta_2, \Delta_3$ se calculan de la siguiente manera

$$ 
\begin{bmatrix}
\frac{1}{2} & -\frac{1}{4} & 0 \\
0 & \frac{1}{2} & 0 \\
-2 & 1 & 1 \\
\end{bmatrix}
\begin{bmatrix}
430 + \Delta_1 \\
460 \\
420
\end{bmatrix} =
\begin{bmatrix}
100 + \frac{1}{2} \Delta_1 \\
230 + 0\\
20 - 2 \Delta_1 \\
\end{bmatrix}  \geq 
\begin{bmatrix}
0 \\
0\\
0
\end{bmatrix}
$$


In [203]:
delta1 = [-200, 10]
b1_lower = right_hand_side + [delta1[0], 0, 0]
b1_upper = right_hand_side + [delta1[1], 0, 0]

print(Binv.dot(b1_lower))
print(Binv.dot(b1_upper))

[0 230 420]
[105 230 0]


In [204]:
delta2 = [-20, 400]
b2_lower = right_hand_side + [0, delta2[0], 0]
b2_upper = right_hand_side + [0, delta2[1], 0]

print(Binv.dot(b2_lower))
print(Binv.dot(b2_upper))

[105 220 0]
[0 430 420]


In [205]:
delta3 = [-20, 1000000]
b3_lower = right_hand_side + [0, 0, delta3[0]]
b3_upper = right_hand_side + [0, 0, delta3[1]]

print(Binv.dot(b3_lower))
print(Binv.dot(b3_upper))

[100 230 0]
[100 230 1000020]


Suponga que los coeficientes del lado izquierdo (coeficientes tecnológicos) de  la columna $x_1$ cambian de $[1 \;,\; 3 \;,\; 1]^{T}$ a $[1 \;,\; 1 \;,\; 6]^{T}$ en la matriz inicial y los coeficientes de $x_2$ and $x_3$ en la función objetivo cambian de $[2\;, \; 5]$ a $[1\;,\; 3]$ **encuentre la nueva solución óptima.**

In [206]:
P1 = np.array([1, 1, 6])


$$\begin{align*}
\pi & = c_b B^{-1} \\[3mm]
z_j &= \pi P_j\\[3mm]
\overline{P_j} &= B^{-1} P_j\\[3mm]
\end{align*}$$

$$\overline{c_j}  = c_j - z_j = c_j - \pi P_j$$

In [207]:
print(zvector)

[3 2 5 0 0 0]


In [208]:
zvector2 = np.array([3, 1, 3, 0, 0,0])
Pbar1 = Binv.dot(P1)
print(Pbar1)


[1/4 1/2 5]


In [209]:
print(table.columns)

Index(['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'b', 'cb', 'cbidx'], dtype='object')


In [210]:
A2 = table.loc[:, 'x1':'x6']
print(A2)
A2['x1'] = Pbar1
A2 = A2.to_numpy()
print(A2)

      x1   x2   x3   x4    x5   x6
x2 -0.25  1.0  0.0  0.5 -0.25  0.0
x3  1.50  0.0  1.0  0.0  0.50  0.0
x6  2.00  0.0  0.0 -2.0  1.00  1.0
[[1/4 1 0 1/2 -1/4 0]
 [1/2 0 1 0 1/2 0]
 [5 0 0 -2 1 1]]


In [229]:
lastrows = last_rows(cj=zvector2, cb=zvector2[cbindex], matrix=A2, label='x')

print(lastrows)

           x1   x2   x3   x4    x5   x6
zj       1.75  1.0  3.0  0.5  1.25  0.0
cj - zj  1.25  0.0  0.0 -0.5 -1.25  0.0


In [232]:
print(np.all(lastrows.iloc[1] <= 0))

False


In [258]:
table2 = simplex(matrix=A2, rhs=solution['b'].values, z=zvector2)

========== Simplex Method ==========
Optimal solution found in 1 iterations
Optimal Table: 
 [[0 1 0 3/5 -3/10 -1/20]
 [0 0 1 1/5 2/5 -1/10]
 [1 0 0 -2/5 1/5 1/5]] 

Solution [4 99 228 0 0 0] 	Z: 795.00
========== Simplex Method ==========


In [259]:
print(table2)

     x1   x2   x3   x4   x5    x6      b   cb  cbidx
x2  0.0  1.0  0.0  0.6 -0.3 -0.05   99.0  1.0    1.0
x3  0.0  0.0  1.0  0.2  0.4 -0.10  228.0  3.0    2.0
x1  1.0  0.0  0.0 -0.4  0.2  0.20    4.0  3.0    0.0
